In [108]:
%%capture
%run 01_transaction.ipynb

In [109]:
from eth_account.messages import encode_defunct as encode_msg
import web3 as w3; w3 = w3.Account

Create a public key `pub_key`, private key `priv_key` pair. The public key is used as an address and the private key is used for signing txs.

In [110]:
def keys():
    acc = w3.create()
    return acc.privateKey.hex(), acc.address

priv_key,pub_key = keys()
ph(priv_key), ph(pub_key)

('🕉 0xe349c60ea3bed8...eed', '💂 0x1C948871282c53...02f')

## Account

Holds a key pair and is used for signing txs.

In [111]:
class Account:
    def __init__(self): 
        self.priv, self.pub = keys()
        self.nonce = 0
        
    def sign(self, tx):
        self.nonce += 1
        m = encode_msg(bytes(tx))
        sig = w3.sign_message(m, self.priv)
        tx.sig = sig
        return tx
            
    def signed_tx(self, to, value):
        tx = TX(self.pub, to.pub, value, self.nonce)
        return self.sign(tx)
    
    def __str__(self): return ph(self.pub)

In [112]:
acc1 = Account(); print(acc1)
acc2 = Account(); print(acc2)

📑 0x6b5f46429b142E...A9F
📴 0x8e5503693BdaEC...969


### Transaction Signing
Signing a tx generates a signature that only the private key can produce. This can be later validated very efficiently.

In [113]:
tx        = TX(acc1.pub, acc2.pub, 12, acc1.nonce)
tx_signed = acc1.sign(tx)

The signature contains the tx hash `message_hash` and its `signature`. We don't need `r`, `v` and `s` as they are specific to Ethereum. For more infos: https://medium.com/@angellopozo/ethereum-signing-and-validating-13a2d7cb0ee3

In [114]:
tx_signed.sig

SignedMessage(messageHash=HexBytes('0x0cad07098f89a4f083802826a48fe294316be1bf22c6bc193196b07c615363ae'), r=49073828320186618201722256979268210358271732729875717328534344656354003280690, s=20521847008405184128372246028773805034481954321499039086465027389676199063387, v=27, signature=HexBytes('0x6c7ecd726e1e29491d3d503e3d1920c946652545b7e14b31f3742eb64a8a63322d5ef42b786df2a90c3608300617c1482ac27e8f6ad957e0c5b2be89c5bfaf5b1b'))

### Transaction Validation

Validates if the tx was signed by the sender.

In [115]:
def val_tx(tx):
    if not hasattr(tx, 'sig'): return False
    m = encode_msg(bytes(tx))
    return w3.recover_message(m, signature=tx.sig.signature) == tx.fr

In [116]:
assert val_tx(tx_signed)

If anything in the tx is changed, like increasing the value to send, the signature should become invalid.

In [117]:
tx_signed.value = 30
assert not val_tx(tx_signed)